In [13]:
!pip install transformers datasets evaluate torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)




Using device: cuda


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate


In [4]:
dataset = load_dataset("imdb")  # IMDB movie reviews
print(dataset)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [5]:
a = dataset['unsupervised'][0]
a



{'text': 'This is just a precious little diamond. The play, the script are excellent. I cant compare this movie with anything else, maybe except the movie "Leon" wonderfully played by Jean Reno and Natalie Portman. But... What can I say about this one? This is the best movie Anne Parillaud has ever played in (See please "Frankie Starlight", she\'s speaking English there) to see what I mean. The story of young punk girl Nikita, taken into the depraved world of the secret government forces has been exceptionally over used by Americans. Never mind the "Point of no return" and especially the "La femme Nikita" TV series. They cannot compare the original believe me! Trash these videos. Buy this one, do not rent it, BUY it. BTW beware of the subtitles of the LA company which "translate" the US release. What a disgrace! If you cant understand French, get a dubbed version. But you\'ll regret later :)',
 'label': -1}

In [6]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "label"])


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

In [9]:
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)


In [10]:
training_args = TrainingArguments(
    output_dir="./sentiment_model",      # jaha model save hoga
    eval_strategy="epoch",          # har epoch evaluate kare
    save_strategy="epoch",                # har epoch checkpoint save kare
    learning_rate=2e-5,                   # optimizer learning rate
    per_device_train_batch_size=16,       # GPU memory ke hisaab se batch
    per_device_eval_batch_size=16,
    num_train_epochs=2,                   # 2 epochs → fast demo
    weight_decay=0.01,                    # regularization
    logging_dir="./logs",                 # tensorboard logs
    logging_steps=50,                     # logs har 50 steps
    load_best_model_at_end=True ,        # best eval model save
    report_to=["none"]
)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-2147312783.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.312300,0.309657,0.868400
2,0.199100,0.321653,0.878680


TrainOutput(global_step=3126, training_loss=0.2874982181605207, metrics={'train_runtime': 721.9106, 'train_samples_per_second': 69.261, 'train_steps_per_second': 4.33, 'total_flos': 1655842483200000.0, 'train_loss': 0.2874982181605207, 'epoch': 2.0})

In [14]:
results = trainer.evaluate()
print(results)


{'eval_loss': 0.3096567690372467, 'eval_accuracy': 0.8684, 'eval_runtime': 91.0919, 'eval_samples_per_second': 274.448, 'eval_steps_per_second': 17.158, 'epoch': 2.0}


In [16]:
model.save_pretrained("./sentiment_model_final")
tokenizer.save_pretrained("./sentiment_model_final")


('./sentiment_model_final/tokenizer_config.json',
 './sentiment_model_final/special_tokens_map.json',
 './sentiment_model_final/vocab.txt',
 './sentiment_model_final/added_tokens.json',
 './sentiment_model_final/tokenizer.json')

In [21]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# 1️⃣ Human-readable labels
id2label = {0: "NEGATIVE", 1: "POSITIVE"}

# 2️⃣ Load fine-tuned model
model_name = "./sentiment_model_final"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 3️⃣ Set id2label for easy mapping
model.config.id2label = id2label

# 4️⃣ Create pipeline
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# 5️⃣ Predict
texts = [
    "I loved this movie! Amazing and fun.",                # Positive
    "Terrible movie, waste of time.",                      # Negative
    "The acting was decent, but the plot was boring.",    # Negative
    "Absolutely fantastic! I would watch it again.",      # Positive
    "Mediocre storyline, but great visuals.",             # Neutral-ish → model might say POSITIVE
    "Worst film I have seen this year.",                  # Negative
    "An enjoyable experience with some minor flaws."      # Positive
]

results = classifier(texts)

for r in results:
    if r['score'] >= 0.9:
        print(r)  # Confident prediction (POSITIVE/NEGATIVE)
    else:
        # Less confident → mark as NEUTRAL
        r['label'] = 'NEUTRAL'
        print(r)


Device set to use cuda:0


{'label': 'POSITIVE', 'score': 0.9900785684585571}
{'label': 'NEGATIVE', 'score': 0.9930693507194519}
{'label': 'NEGATIVE', 'score': 0.9907016754150391}
{'label': 'POSITIVE', 'score': 0.9696831703186035}
{'label': 'NEUTRAL', 'score': 0.7323464751243591}
{'label': 'NEGATIVE', 'score': 0.9918151497840881}
{'label': 'POSITIVE', 'score': 0.9877730011940002}


"An industry-ready sentiment analysis pipeline was developed using Hugging Face Transformers. A DistilBERT model was fine-tuned on the IMDB movie review dataset to accurately predict review sentiments. Training and evaluation were conducted on a GPU-enabled environment, with tokenization and attention masks applied. Score thresholds were implemented to identify low-confidence predictions, ensuring robust and reliable outputs. This workflow demonstrates a professional fine-tuning approach and can be easily extended to tasks such as spam detection, summarization, and zero-shot classification."